In [4]:
import os
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd



@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    target_column:str

In [6]:
from src.utils.utlis import read_yaml,create_dir,save_json
from src.constants import *

In [7]:
class ConfigManager:
    def __init__(self,
                  config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_dir([self.config.artifacts_root])

    def model_evluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema =  self.schema.TARGET_COLUMN

        create_dir([config.root_dir]) 

        model_Eval_Config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            target_column=schema.name
        )   

        return model_Eval_Config

In [8]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score,f1_score,precision_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        acc = np.sqrt(accuracy_score(actual, pred))
        f1 = f1_score(actual, pred, average='macro')
        precision = precision_score(actual, pred, average='macro')

        return acc,f1,precision
    
    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)

        (acc, f1, precision) = self.eval_metrics(test_y, predicted_qualities)
        
        # Saving metrics as local
        scores = {"Accuracy": acc, "F1_score": f1, "Precision": precision}
        save_json(path=Path(self.config.metric_file_name), data=scores)


In [10]:
try:
    config = ConfigManager()
    model_evaluation_config = config.model_evluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

/Users/akashmukherjee/Programming/MLOPS/Bappy/Wine quality/my_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
